In [ ]:
# Header starts here.
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
from copy import deepcopy
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        1,              #  1
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    objc = deepcopy(obj)
    try:
        tmp = DX(str(float(objc)))
        objc = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(objc)):
            tmp = DX(str(float(objc[i])))
            objc[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return objc

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm) = ( m/1000, m/100 )
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

half = S(1)/2

# Header ends here.
#
# https://colab.research.google.com/github/kassbohm/tm-snippets/blob/master/ipynb/TM_2/4_BB/2_BL/2.4.2.I-FEM_cc.ipynb

a, EI, F, q = var('a, EI, F, q')

sub_list = [
    (a, 3 *m),
    (q, 3 *kN/m),
    (EI, 200 *GPa* 6500 *cm**4),
    (F, 10 *kN),
    ]

a2 = a*a
a3 = a*a*a

K44 = EI/a3
K44 *= Matrix(
[
[  4*a2 ,  -6*a ,  2*a2 ,   6*a ],
[ -6*a  ,  12   , -6*a  , -12   ],
[  2*a2 ,  -6*a ,  4*a2 ,   6*a ],
[  6*a  , -12   ,  6*a    , 12   ],
]
)
# pprint(K/EI*a**3)

K = K44[0:2, 0:2]

p1,w1 = var("p1,w1")
M2,F2 = var("M2,F2")
u = Matrix([p1,w1])
f = Matrix([0, F]) + q*Matrix([-a2/12, a/2])
unknowns = [p1, w1]
eq = Eq(K*u , f)
sol = solve(eq, unknowns)

p1, w1 = sol[p1], sol[w1]

pprint("\nphi1 / deg:")
tmp = p1
tmp = tmp.subs(sub_list)
tmp *= 180/pi
tmp = iso_round(tmp, 0.01)
pprint(tmp)

pprint("\nw1 / mm:")
tmp = w1
tmp = tmp.subs(sub_list)
tmp /= mm
tmp = iso_round(tmp, 0.01)
pprint(tmp)

# ---

K = K44[2:, 0:2]
u = Matrix([p1,w1])
f = Matrix([M2, F2]) + q*Matrix([a2/12, a/2])
unknowns = [M2, F2]
eq = Eq(K*u , f)
pprint(eq)
sol = solve(eq, unknowns)

M2, F2 = sol[M2], sol[F2]

pprint("\nM2:")
tmp = M2
pprint(tmp)
pprint("\nM2 / kNm:")
tmp = tmp.subs(sub_list)
tmp /= k*Newton*m
pprint(tmp)

pprint("\nF2:")
tmp = F2
pprint(tmp)
pprint("\nF2 / kN:")
tmp = tmp.subs(sub_list)
tmp /= kN
pprint(tmp)

# phi1 / deg:
# 0.26
#
# w1 / mm:
# 9.26
#                                     ⎡      2  ⎤
# ⎡      ⎛F   a⋅q⎞   a⋅(3⋅F + a⋅q)⎤   ⎢     a ⋅q⎥
# ⎢- 6⋅a⋅⎜─ + ───⎟ + ─────────────⎥   ⎢M₂ + ────⎥
# ⎢      ⎝3    8 ⎠         3      ⎥   ⎢      12 ⎥
# ⎢                               ⎥ = ⎢         ⎥
# ⎢                a⋅q            ⎥   ⎢     a⋅q ⎥
# ⎢           -F - ───            ⎥   ⎢F₂ + ─── ⎥
# ⎣                 2             ⎦   ⎣      2  ⎦
#
# M2:
# -a⋅(2⋅F + a⋅q)
# ───────────────
#        2
#
# M2 / kNm:
# -87/2
#
# F2:
# -F - a⋅q
#
# F2 / kN:
# -19
